In [1]:
""" Installation of library are mentioned here """
!pip install happytransformer 
from IPython.display import clear_output
clear_output()

In [1]:
import csv
from datasets import load_dataset
from happytransformer import TTSettings
from happytransformer import TTTrainArgs
from happytransformer import HappyTextToText

/u/02/raya1/unix/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.1) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/u/02/raya1/unix/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-04 01:29:51.386010: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environ

In [20]:
happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")

05/04/2023 01:36:19 - INFO - happytransformer.happy_transformer -   Using model: cuda


In [3]:
import pandas as pd
df = pd.read_csv("testing_data_cleaned.csv")
# df
# fill in the values of col5 into col6 
df['col6'].fillna(df['col5'], inplace=True)
X, y = df['col5'].values, df['col6'].values

# train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8297,), (922,), (8297,), (922,))

In [4]:
def generate_csv(csv_path, X, y):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])

        for i in range(X.shape[0]) :

            input_text = "grammar: " + X[i]
            correction = y[i]

            # a few of the cases contain blank strings. 
            if input_text and correction:
                writter.writerow([input_text, correction])


In [5]:
generate_csv("eval.csv", X_test, y_test)
generate_csv("train.csv", X_train, y_train)

In [21]:
before_result = happy_tt.eval("eval.csv")

05/04/2023 01:36:30 - INFO - happytransformer.happy_transformer -   Preprocessing evaluating data...


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 177.79it/s]


Dataset csv downloaded and prepared to /u/02/raya1/unix/.cache/huggingface/datasets/csv/default-b53a2259e1e84363/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


Map:   0%|          | 0/922 [00:00<?, ? examples/s]/u/02/raya1/unix/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [22]:
print("Before loss:", before_result.loss)

Before loss: 0.8561544418334961


In [25]:
# read in eval.csv
text = "I and my friend is going to the park."
print("Original text:", text)

# top_k_sampling_args = TTSettings(do_sample=True, top_k=120, top_p=0.95, early_stopping=True, min_length=1, max_length=30)
input_text = "grammar: " + text + " </s>"
beam_args = TTSettings(num_beams=5, min_length=1, max_length=len(input_text.split()) + 10 , early_stopping=True)
result = happy_tt.generate_text(input_text, args=beam_args)
print("Top k sampling result:", result.text)


Original text: I and my friend is going to the park.
Top k sampling result: I and my friend are going to the park.


In [28]:
# iterate through the sentences
tp = tn = fp = fn = 0
for i, sent in enumerate(X_test) :
    if i%30==0:
        print('i:',i)
    input_text = "grammar: " + sent + " </s>"
    beam_args = TTSettings(num_beams=5, min_length=1, max_length=len(input_text.split()) + 10 , early_stopping=True)
    result = happy_tt.generate_text(input_text, args=beam_args)
    if result.text == sent:
        tn += 1
    else:
        if sent != y_test[i]:
            fn += 1
        else:
            fp += 1


i: 0


/u/02/raya1/unix/.local/lib/python3.8/site-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


i: 30
i: 60
i: 90
i: 120
i: 150
i: 180
i: 210
i: 240
i: 270
i: 300
i: 330
i: 360
i: 390
i: 420
i: 450
i: 480
i: 510
i: 540
i: 570
i: 600
i: 630
i: 660
i: 690
i: 720
i: 750
i: 780
i: 810
i: 840
i: 870
i: 900
True Positive 7375
True Negative 29
False Positive 408
False Positive 485


In [33]:

tp =len(X_test) - tn  - fn
print('True Positive', tp)
print('True Negative', tn)
print('False Positive', fn)
print('False Positive', fp)
precision = tp/(tp+fp)
recall = tp/(tp+fn)
F = (1+0.5**2)*((precision*recall)/((0.5**2 * precision)+recall))
F

True Positive 485
True Negative 29
False Positive 408
False Positive 485


0.5080662057406243

In [34]:
# The cat sat at mat
# Giant otters is an apex predator
# I and my friend is going to the park.

sentences = [
    "The cat sat at mat",
    "Giant otters is an apex predator",
    "I and my friend is going to the park."
]

for sent in sentences :
    input_text = "grammar: " + sent + " </s>"
    beam_args = TTSettings(num_beams=5, min_length=1, max_length=len(input_text.split()) + 10 , early_stopping=True)
    result = happy_tt.generate_text(input_text, args=beam_args)
    print("Original text:", sent)
    print("Top k sampling result:", result.text)

/u/02/raya1/unix/.local/lib/python3.8/site-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Original text: The cat sat at mat
Top k sampling result: The cat sat at matt.
Original text: Giant otters is an apex predator
Top k sampling result: Giant otters are an apex predator.
Original text: I and my friend is going to the park.
Top k sampling result: I and my friend are going to the park.
